# 000 Forecasting Bot

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IFP import IFP, list_questions

In [3]:
ifps = list_questions()['results']

In [4]:
today_ids = list(sorted([x['id'] for x in ifps])) if ifps else [27238, 27239, 27240, 27241]
today_ids

[27920, 27932, 27933, 27934, 27935, 27979, 27980, 27981, 27982, 27983]

In [5]:
ifps = {id: IFP(id) for id in today_ids}

In [6]:
from Forecaster import Forecaster

In [7]:
fcst = Forecaster(ifps)

In [8]:
fcst.fit()

MODEL DOMAIN
27920 Before October 1, 2024, will Ethiopia and Somalia announce an agreement settling their dispute over the Somaliland port deal?
Geopolitics
27932 Will China's youth unemployment rate be less than or equal to 16.0 for August 2024?
Rate change
27933 Will China's youth unemployment rate be greater than 16.0 and less than or equal to 17.0 for August 2024?
Rate change
27934 Will China's youth unemployment rate be greater than 17.0 and less than or equal to 18.0 for August 2024?
Rate change
27935 Will China's youth unemployment rate be greater than 18.0 for August 2024?
Rate change
27979 Will Starliner Calypso touch ground at any of the planned sites by September 30?
Aerospace technology
27980 Will Starliner Calypso touch ground somewhere other than at one of the planned sites by September 30?
Aerospace technology
27981 Will Starliner Calypso splashdown in any body of water by September 30?
Aerospace technology
27982 Will Starliner Calypso disintegrate in the atmosphere by S

In [9]:
for ifp in ifps.values():
    try:
        print(ifp.id, ifp.forecast)
    except:
        print(ifp.id, 'none')

27920 25
27932 2
27933 15
27934 60
27935 25
27979 95
27980 5
27981 1
27982 1
27983 1


In [10]:
fcst.report()

27920 Before October 1, 2024, will Ethiopia and Somalia announce an agreement settling their dispute over the Somaliland port deal?
Forecast 25
Rationale Given the complex geopolitical situation and the ongoing diplomatic talks, it seems unlikely that Ethiopia and Somalia will reach an agreement before the October 1, 2024, deadline. There's significant tension between the parties involved, and although progress has been made in discussions, a resolution by the specified date appears improbable. The third round of talks is scheduled for September 17, 2024, but considering the contentious nature of the dispute and the short time frame, reaching a settlement within approximately two weeks is doubtful. 

27932 Will China's youth unemployment rate be less than or equal to 16.0 for August 2024?
Forecast 2
Rationale The likelihood of China's youth unemployment rate being less than or equal to 16.0% for August 2024 is very low given the recent surge to 17.1% in July 2024. The economic data ind

In [11]:
fcst.upload()

Prediction posted for  27920
Comment posted for  27920
Prediction posted for  27932
Comment posted for  27932
Prediction posted for  27933
Comment posted for  27933
Prediction posted for  27934
Comment posted for  27934
Prediction posted for  27935
Comment posted for  27935
Prediction posted for  27979
Comment posted for  27979
Prediction posted for  27980
Comment posted for  27980
Prediction posted for  27981
Comment posted for  27981
Prediction posted for  27982
Comment posted for  27982
Prediction posted for  27983
Comment posted for  27983
